In [5]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from PIL import Image, ImageDraw
import imutils
import tensorflow
import os
import cv2
import numpy as np

In [6]:
TRAIN_DIR = r"D:\Sign Language Recognization using CNN\New folderr\Gestures"  # Gestures Images at Local machine

num_classes=24        
IMG_SIZE = 100
def vectorize_data(TRAIN_DIR):
    result = []
    labels = []
    for label in os.listdir(TRAIN_DIR):
        path=""
        path=os.path.join(TRAIN_DIR, label)
        for img in os.listdir(path):
            path2=""
            path2 = os.path.join(path, img)
            i = cv2.imread(path2)
            
            i = cv2.resize(cv2.imread(path2, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
            
            
            result.append(i)
            labels.append(label)
    
    return result, labels

x, y =vectorize_data(TRAIN_DIR)
x_train = np.array(x)
y_train = np.array(y)

x_train = np.expand_dims(x_train, axis=-1)
x_train.shape

(624, 100, 100, 1)

In [7]:
from tensorflow.keras.utils import to_categorical

dictonary = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'K':9, 'L':10, 'M':11, 'N':12, 'O':13, 'P':14, 
            'Q':15, 'R':16, 'S':17, 'T':18, 'U':19, 'V':20, 'W':21, 'X':22, 'Y':23}
num_classes=24
keys, inv = np.unique(y_train, return_inverse=True)
vals = np.array([dictonary[key] for key in keys])
y_train_new = vals[inv]
y_train_new_cat = to_categorical(y_train_new, num_classes)

def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]
x_new,y_new = unison_shuffled_copies(x_train,y_train_new_cat)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adadelta
IMG_SIZE = 100

num_classes = 24
model = Sequential()

model.add(Conv2D(16, (2,2), input_shape=(IMG_SIZE, IMG_SIZE, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2) ))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2) ))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2) ))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=SGD(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(x_new, y_new, epochs = 30, validation_split = 0.1, shuffle = True, batch_size = 2)
model.save("Rohit_1.model")

Epoch 1/30
281/281 [==============================] - 7s 23ms/step - loss: 5.4728 - accuracy: 0.0998 - val_loss: 2.8900 - val_accuracy: 0.2063
Epoch 2/30
281/281 [==============================] - 6s 23ms/step - loss: 2.7110 - accuracy: 0.1996 - val_loss: 2.7602 - val_accuracy: 0.2063
Epoch 3/30
281/281 [==============================] - 11s 39ms/step - loss: 2.3208 - accuracy: 0.2834 - val_loss: 2.7589 - val_accuracy: 0.1905
Epoch 4/30
281/281 [==============================] - 7s 24ms/step - loss: 2.0174 - accuracy: 0.3868 - val_loss: 2.2694 - val_accuracy: 0.4127
Epoch 5/30
281/281 [==============================] - 9s 32ms/step - loss: 1.7831 - accuracy: 0.4813 - val_loss: 2.1418 - val_accuracy: 0.4286
Epoch 6/30
281/281 [==============================] - 7s 24ms/step - loss: 1.5223 - accuracy: 0.5544 - val_loss: 2.0946 - val_accuracy: 0.4286
Epoch 7/30
281/281 [==============================] - 8s 30ms/step - loss: 1.2598 - accuracy: 0.6185 - val_loss: 2.1839 - val_accuracy: 0.444

In [10]:
camera = cv2.VideoCapture(0)
model = load_model("Rohit_1.model")
IMG_SIZE = 100
top, right, bottom, left = 10, 350, 225, 590
_,first_frame = camera.read()
SAVE_PATH = "C:/Users/Admin/Downloads/Compressed"
first_gray = cv2.flip(first_frame, 1)
roi = first_gray[top:bottom, right:left]
img_count = 1
dictonary = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'K':9, 'L':10, 'M':11, 'N':12, 'O':13, 'P':14, 
            'Q':15, 'R':16, 'S':17, 'T':18, 'U':19, 'V':20, 'W':21, 'X':22, 'Y':23}
num_class = 24
out = ""
inv_dictonary = dict(map(reversed, dictonary.items()))
string = ""

while True:
    
    _, frame = camera.read()
    
    
    gray_frame = cv2.flip(frame, 1) 
    
    
    roi2 = gray_frame[top:bottom, right:left] 
    cv2.rectangle(gray_frame, (300, 70), (600, 250), (0,255,0), 2) 
    
    diff = cv2.absdiff(roi, roi2)
    diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    diff = cv2.GaussianBlur(diff, (11,11),0)
    _,diff = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    cv2.imshow("Diff", diff)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
    if key == ord('q'):
        break
    if key == ord('r'):
        roi = gray_frame[top:bottom, right:left]
        string = ""
        
    img = cv2.resize(diff, (IMG_SIZE, IMG_SIZE))
    img = tensorflow.cast(img, tensorflow.float32)

    img_arr = np.array(img)

    t2 = np.expand_dims(img_arr, axis=-1)
    t2 = np.expand_dims(t2, axis=0)
    p = model.predict(t2)
    p2 = np.argmax(p)
    out = inv_dictonary[p2]
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(gray_frame,out, (22,34), font, 1, (200,255,255), 2, cv2.LINE_AA)
    cv2.putText(gray_frame,string, (22,64), font, 1, (200,255,255), 2, cv2.LINE_AA)
    cv2.putText(gray_frame,"r: Reset;  s: Append;  q: Quit", (22,470), font, 1, (200,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow("Frame", gray_frame) #gray_frame
    if key == ord('s'):
        string = string+""+out
        
    if key == ord('c'):
        img_name = "data{}.png".format(img_count)
        img_count+=1
        cv2.imwrite(os.path.join(SAVE_PATH, img_name), diff)
        
        img_name = "data{}.png".format(img_count)
        img_count+=1
        cv2.imwrite(os.path.join(SAVE_PATH, img_name), gray_frame)
        
        img_name = "data{}.png".format(img_count)
        img_count+=1
        cv2.imwrite(os.path.join(SAVE_PATH, img_name), roi2)
   
camera.release()
cv2.destroyAllWindows()